# Derive Top N Portfolio

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [35]:
import os
import sys
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
# import rezaware as reza
from utils.modules.etl.load import sparkNoSQLwls as nosql
# from utils.modules.etl.load import sparkDBwls as sdb
# from utils.modules.etl.transform import sparkCleanNRich as scne
from mining.modules.assets.etp import dailyTopN as topN
# from utils.modules.ml.timeseries import rollingstats as stats

''' restart initiate classes '''
if debug:
    import importlib
    topN = importlib.reload(topN)
    nosql= importlib.reload(nosql)
    
__desc__ = "analyze crypto market capitalization time series data"
clsMPT =topN.WeightedPortfolio(desc=__desc__)

print("\nClass initialization and load complete!")

All functional DAILYTOPN-libraries in ETP-package of ASSETS-module imported successfully!
All functional SPARKNOSQLWLS-libraries in LOAD-package of ETL-module imported successfully!
sparkNoSQLwls Class initialization complete
sparkNoSQLwls Class initialization complete
execSession Class initialization complete
performIndex Class initialization complete
execSession Class initialization complete
dailyTopN Class initialization complete

Class initialization and load complete!


## Build the Weighted Portfolio
We apply a query to select assets with mcap > 1.0 million. Any missing values are imputed with the mean value.

### Write MPT to MongoDB

* Collection name = "mpt."+date(YYYY-MM-DD)
* document structure: \_id, date, asset, mcap.value, mcap.weight, mcap.ror

In [36]:
_date = date(2022,1,27)
_portfolio, _ror_data = clsMPT.build_weighted_portfolio(
    mcap_date=_date,
    pick_asset_db_name=None,
    ror_db_name=None,
)

Total 1 documents, successful insert count = 0 & modify count = 1


## Display results

In [37]:
from pyspark.sql import functions as F

_ror_data.select(
    F.col('mcap_past_pk'),F.col('mcap_date'),
    F.col('asset_name'),F.col('price_value'),F.col('price_log_ror'))\
    .sort(F.col('price_date'))\
    .show(n=10)

print("weighted portfolio\n",_portfolio)

+------------+-------------------+----------+-----------+-------------+
|mcap_past_pk|          mcap_date|asset_name|price_value|price_log_ror|
+------------+-------------------+----------+-----------+-------------+
|      158144|2022-01-27 00:00:00|      wbtc|       null|         null|
+------------+-------------------+----------+-----------+-------------+

weighted portfolio
 [[{'date': '2022-01-27T00:00:00', 'asset': 'wbtc', 'mcap.weight': 1.0, 'mcap.ror': 0.6931471805599453, 'mcap.value': 8021359.81238079, 'mcap.db.fk': '158144', 'mcap.db.source': 'warehouse.mcap_past', 'audit.mod.by': 'NUWAN', 'audit.mod.dt': '2023-04-20T13:59:28', 'audit.mod.proc': 'dailyTopN-function <write_mpt_to_db>'}]]
